In [1]:
SEED=185

In [2]:
import pandas as pd
import numpy as np

from runModels import runModels

from sklearn import preprocessing
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [3]:
DATA='data/mamografias.csv'

# Función para leer los datos
def readData(data_file):
    return pd.read_csv(data_file,sep=',', na_values='?')

data = readData(DATA) # Lectura de los datos

data.rename(columns = {'BI-RADS':'BiRads'}, inplace = True) # Para poder referirnos a esta columna como data.BiRads
data.BiRads.replace(0,pd.NA,inplace=True) # BiRads 0 significa radiografía insuficiente
data.Shape.replace('N',pd.NA,inplace=True) # Lo mismo pasa con Shape N
data.replace(pd.NA, np.nan, inplace=True)

data=data.dropna() # Eliminamos las instancias con valores perdidos
data.shape[0] # Nos quedamos con 825 instancias

# La distribución de la característica densidad es prácticamente degenerada en 3
data.drop('Density', axis='columns',inplace=True)
data.BiRads=data.BiRads.replace(6,5).replace(2,4).replace(3,4)

# Sklearn necesita datos numéricos (aunque sean nominales)
le = preprocessing.LabelEncoder()
data.Shape = le.fit_transform(data.Shape)
data.Severity = le.fit_transform(data.Severity)

dataArray=np.array(data)

# Separamos en datos y target (label)
x=dataArray[:,:-1]
y=dataArray[:,-1]

# Binarización de los atributos nominales
x_nom=x[:,2:4] # Columnas con variables nominales: Shape y Margin
x=np.delete(x,[2,3],axis=1) # Elimino las columnas
onehot = preprocessing.OneHotEncoder(sparse=False)
x_nom = onehot.fit_transform(x_nom)
x=np.hstack((x,x_nom))
# Para los árboles no reescalo los atributos porque no tiene efecto

In [4]:
# Árbol de decisión con distintos valores del parámetro depth
dt=[tree.DecisionTreeClassifier(max_depth=depth, random_state=SEED) for depth in [3,6,9]]
model_names=['DT-depth'+str(d) for d in [3,6,9]]

In [5]:
runModels(x,y,'DT_tuning',dt,model_names)

In [6]:
# Random Forest con distinto número de estimadores
rf=[RandomForestClassifier(n_estimators=n, random_state=SEED) for n in [50,100,150,200]]
model_names=['RF-estimators'+str(n) for n in [50,100,150,200]]

In [7]:
runModels(x,y,'RF_tuning',rf,model_names)

In [8]:
# Random Forest con distinto número de estimadores
rf=[RandomForestClassifier(n_estimators=n, random_state=SEED) for n in [400,600,800,1000]]
model_names=['RF-estimators'+str(n) for n in [400,600,800,1000]]

In [9]:
runModels(x,y,'RF_tuning2',rf,model_names)